<a href="https://colab.research.google.com/github/RussiANC00lHACKA/StartML/blob/main/iris_csv_prac.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
import numpy as np

url = 'https://gist.githubusercontent.com/RussiANC00lHACKA/6cbc3923b548d3d0f17be9914ba75772/raw/821e5bd3e97765faf79e0d0aa97e3360b70cf61a/iris.csv'

data = pd.read_csv(url)
data.head()

,sepal.length,sepal.width,petal.length,petal.width,variety
0,5.1,3.5,1.4,0.2,Setosa
1,4.9,3.0,1.4,0.2,Setosa
2,4.7,3.2,1.3,0.2,Setosa
3,4.6,3.1,1.5,0.2,Setosa
4,5.0,3.6,1.4,0.2,Setosa


In [6]:
data['variety'].value_counts()

Setosa        50
Versicolor    50
Virginica     50
Name: variety, dtype: int64

In [11]:
data['variety'] = data['variety'].map({'Setosa':1, 'Versicolor':2, 'Virginica':3})

In [12]:
data.head()

,sepal.length,sepal.width,petal.length,petal.width,variety
0,5.1,3.5,1.4,0.2,1
1,4.9,3.0,1.4,0.2,1
2,4.7,3.2,1.3,0.2,1
3,4.6,3.1,1.5,0.2,1
4,5.0,3.6,1.4,0.2,1


In [28]:
#проверим точность работы DecisionTree и Knn для данного дата сета
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split, cross_val_score,GridSearchCV

In [18]:
#разобьем данный датасет на подсеты под холдаут и кросс-валидацию
x = data.drop('variety',axis = 1)
y = data['variety']
x_train,x_valid,y_train,y_valid = train_test_split(x,y,train_size = 0.3, random_state=17)

In [19]:
x_train.shape,x_valid.shape,y_train.shape,y_valid.shape

((45, 4), (105, 4), (45,), (105,))

In [27]:
ft = DecisionTreeClassifier(random_state = 17)
ft.fit(x_train,y_train)
np.mean(cross_val_score(ft,x_train,y_train,cv = 5)) #LOOCV

0.9333333333333332

In [30]:
x_valid

,sepal.length,sepal.width,petal.length,petal.width
16,5.4,3.9,1.3,0.4
78,6.0,2.9,4.5,1.5
145,6.7,3.0,5.2,2.3
99,5.7,2.8,4.1,1.3
126,6.2,2.8,4.8,1.8
...,...,...,...,...
72,6.3,2.5,4.9,1.5
103,6.3,2.9,5.6,1.8
88,5.6,3.0,4.1,1.3
118,7.7,2.6,6.9,2.3


In [31]:
y_valid


16     1
78     2
145    3
99     2
126    3
      ..
72     2
103    3
88     2
118    3
128    3
Name: variety, Length: 105, dtype: int64

In [33]:
ft.score(x_valid,y_valid) #Для холдаута точность

0.9619047619047619

In [37]:
#понастраиваем параметры
t_params = {'max_features':[0.5, 0.7, 1],'max_depth':list(range(1,11)) + list(range(10,100,10))}
t_grid = GridSearchCV(ft, t_params,n_jobs = -1,cv = 5)
t_grid.fit(x_train,y_train)

GridSearchCV(cv=5, estimator=DecisionTreeClassifier(random_state=17), n_jobs=-1,
             param_grid={'max_depth': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 10, 20,
                                       30, 40, 50, 60, 70, 80, 90],
                         'max_features': [0.5, 0.7, 1]})

In [45]:
t_grid.best_score_ #точность которая достигается с лучшими параметрами деревьев

0.9555555555555555

In [40]:
t_grid.best_estimator_

DecisionTreeClassifier(max_depth=2, max_features=0.5, random_state=17)

In [46]:
t_grid.score(x_valid,y_valid) #точность лучшей модели на холдауте

0.9523809523809523

In [49]:
#теперь проверим knn
from sklearn.neighbors import KNeighborsClassifier
fknn = KNeighborsClassifier()
fknn.fit(x_train,y_train)
np.mean(cross_val_score(fknn,x_train,y_train,cv = 5, n_jobs = -1)) #точность на кросс-валидации

0.888888888888889

In [52]:
fknn.score(x_valid,y_valid) #точность на холдауте

0.9714285714285714

In [57]:
#найдем лучшие локальные параметры под модель
fknn_params = {'n_neighbors':list(range(1,20))}
fknn_grid = GridSearchCV(fknn, fknn_params, cv = 5, n_jobs = -1)

In [61]:
fknn_grid.fit(x_train,y_train)

GridSearchCV(cv=5, estimator=KNeighborsClassifier(), n_jobs=-1,
             param_grid={'n_neighbors': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12,
                                         13, 14, 15, 16, 17, 18, 19]})

In [64]:
fknn_grid.best_estimator_

KNeighborsClassifier(n_neighbors=3)

In [62]:
fknn_grid.best_score_#точность лучшей модели knn на кроссвалидации

0.9555555555555555

In [63]:
fknn_grid.score(x_valid,y_valid) #точность лучшей модели на холдауте

0.9619047619047619

In [65]:
# по итогу: acc_knn_cv = 0.9555555555555555, acc_knn_holdout = 0.9619047619047619, acc_dt_cv = 0.9555555555555555, acc_dt_holdout = 0.9523809523809523